# **Disclaimers**
The data preprocessing was done on Google Collab so if you plan to replicate this locally please ignore the collab imports and configure the local dataset locations yourself pls! <br>

Here is a link to our datasets used in this notebook: https://drive.google.com/drive/folders/1Ma-HdWP_xlbLLBu8EGPCFasVftkIPQjt?usp=sharing <br>

**The dataset generation was initially for the whole of Luzon and not just NCR, this you may notice a lot of provinces. Please just ignore it as we extract the NCR rows in the last part of the processin!**

# **Setting up the Env**

In [ ]:
#importing the relevant libraries
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

pd.set_option('display.max_rows', None)

In [ ]:
# Mounting on your google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#%cd /content/drive/MyDrive/Classroom/data
%cd /content/drive/MyDrive/Thesis Stuff/Code Notebooks/data

/content/drive/MyDrive/Thesis Stuff/Code Notebooks/data


In [ ]:
%ls

DOH_Hospitals.csv                       ph_commobility.csv
Feb52022_COVID_Data.csv                 PH_lockdowns_cleanest.csv
Luzon_Incidence_Rates_PopDens_2020.csv  Provinces/
Nov7-COVIDCases.csv                     Sept132021_COVID_Data.csv


# **Covid Cases Section** 
#### Adding the dataframes related to Covid Counts, Deaths, and Recoveries 
#### ```cases_agg_df, df_aggrecov, df_aggredied```

In [ ]:
df = pd.read_csv("May72022_COVID_Data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(3686872, 22)

In [ ]:
df.tail(5)

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
3686867,C28942786,37.0,35 to 39,MALE,2022-05-05,2022-05-05,2022-05-07,NaN,NaN,NaN,...,NCR,CITY OF MANILA,PH133910000,BARANGAY 719,PH133910034,ASYMPTOMATIC,NO,NaN,NaN,NaN
3686868,C29491130,31.0,30 to 34,MALE,2022-05-02,2022-05-02,2022-05-07,NaN,NaN,NaN,...,NCR,CITY OF PARAÑAQUE,PH137604000,MERVILLE,PH137604010,ASYMPTOMATIC,NO,2022-05-02,NaN,NaN
3686869,C15235102,2.0,0 to 4,FEMALE,2022-05-02,2022-05-03,2022-05-07,NaN,NaN,NaN,...,SOUTH COTABATO,NORALA,PH126311000,"BENIGNO AQUINO, JR.",PH126311022,MILD,NO,2022-04-28,NO,NaN
3686870,C65427022,73.0,70 to 74,MALE,2022-01-16,2022-01-17,2022-05-07,NaN,NaN,RECOVERED,...,CAVITE,TANZA,PH042120000,MULAWIN,PH042120012,RECOVERED,NO,2022-01-15,NaN,"Health Status is ""Recovered"", but no Date Reco..."
3686871,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dtypes

CaseCode             object
Age                  object
AgeGroup             object
Sex                  object
DateSpecimen         object
DateResultRelease    object
DateRepConf          object
DateDied             object
DateRecover          object
RemovalType          object
Admitted             object
RegionRes            object
ProvRes              object
CityMunRes           object
CityMuniPSGC         object
BarangayRes          object
BarangayPSGC         object
HealthStatus         object
Quarantined          object
DateOnset            object
Pregnanttab          object
ValidationStatus     object
dtype: object

In [ ]:
# Dropping the rows without dates of confirmed cases as that information is the most important 
df = df.dropna(subset=['DateRepConf'])

In [ ]:
# Converting the DateRepConf (Date - of covid cases - Report Confirmed) from an Object type to a DateTime format
df["DateRepConf"] = pd.to_datetime(df["DateRepConf"], errors='coerce')

## **Section for the Deaths and Recoveries**
####Recovered Cases = df_recov
####Died Peoples = df_died

In [ ]:
# We just need to know the specific case number, when they recovered, and which province they're from
df_recov = pd.concat([df["CaseCode"], df["DateRecover"],df["ProvRes"]], axis=1)
df_died = pd.concat([df["CaseCode"], df["DateDied"],df["ProvRes"]], axis=1)
df_recov.shape + df_died.shape 

(3686871, 3, 3686871, 3)

In [ ]:
#for the recovery cases that had actual dates
conditional1 = (df_recov.DateRecover.isnull() != True)
#for the death cases that had actual dates
conditional2 = (df_died.DateDied.isnull() != True) 

df_recov = df_recov[conditional1]
df_died = df_died[conditional2]

df_recov.shape + df_died.shape 

(663012, 3, 60144, 3)

In [ ]:
df_died.head()

,CaseCode,DateDied,ProvRes
1,C462688,2020-02-01,NEGROS ORIENTAL
4,C498051,2020-03-11,RIZAL
5,C130591,2020-03-12,RIZAL
6,C557002,2020-03-14,NCR
10,C382946,2020-04-09,NCR


In [ ]:
# Obtaining provinces in Luzon (only) as the dataset contains all provinces in the PH
df_recov = df_recov[df_recov["ProvRes"].str.contains("NCR|ABRA|ALBAY|APAYAO|AURORA|BATAAN|BATANGAS|BENGUET|BULACAN|CAGAYAN|CAMARINES NORTE|CAMARINES SUR|CAVITE|IFUGAO|ILOCOS NORTE|ILOCOS SUR|ISABELA|KALINGA|LA UNION|LAGUNA|MOUNTAIN PROVINCE|NUEVA ECIJA|NUEVA VIZCAYA|PAMPANGA|PANGASINAN|QUEZON|QUIRINO|RIZAL|SORSOGON|TARLAC|ZAMBALES")==True]
df_died = df_died[df_died["ProvRes"].str.contains("NCR|ABRA|ALBAY|APAYAO|AURORA|BATAAN|BATANGAS|BENGUET|BULACAN|CAGAYAN|CAMARINES NORTE|CAMARINES SUR|CAVITE|IFUGAO|ILOCOS NORTE|ILOCOS SUR|ISABELA|KALINGA|LA UNION|LAGUNA|MOUNTAIN PROVINCE|NUEVA ECIJA|NUEVA VIZCAYA|PAMPANGA|PANGASINAN|QUEZON|QUIRINO|RIZAL|SORSOGON|TARLAC|ZAMBALES")==True]

df_recov.shape + df_died.shape

(409221, 3, 36250, 3)

In [ ]:
# Converting the dates from Objects into DateTime Formats
df_recov["DateRecover"] = pd.to_datetime(df_recov["DateRecover"])
df_died["DateDied"] = pd.to_datetime(df_died["DateDied"])

In [ ]:
# Aggregating each the number of deaths and recoveries per province then per date
df_aggrecov = df_recov.groupby(["ProvRes", "DateRecover"]).agg(['count']).sort_values(["ProvRes", "DateRecover"], ascending = (True, True))
df_aggrecov = df_aggrecov.reset_index()

df_aggredied = df_died.groupby(["ProvRes", "DateDied"]).agg(['count']).sort_values(["ProvRes", "DateDied"], ascending = (True, True))
df_aggredied = df_aggredied.reset_index()

print(len(df_aggrecov), len(df_aggredied))

10147 9323


In [ ]:
df_aggredied.tail()

,ProvRes,DateDied,CaseCode
,,,count
9318,ZAMBALES,2022-04-07,1
9319,ZAMBALES,2022-04-08,1
9320,ZAMBALES,2022-04-18,1
9321,ZAMBALES,2022-04-21,1
9322,ZAMBALES,2022-04-27,1


## The actual Covid Case Counts per Province (daily aggregation)

In [ ]:
df_new = pd.concat([df["CaseCode"], df["DateRepConf"],df["ProvRes"]], axis=1)

In [ ]:
#These comprise of all provinces in Luzon, Visayas, and Mindanao
df_new.tail()

,CaseCode,DateRepConf,ProvRes
3686866,C32158902,2022-05-07,NCR
3686867,C28942786,2022-05-07,NCR
3686868,C29491130,2022-05-07,NCR
3686869,C15235102,2022-05-07,SOUTH COTABATO
3686870,C65427022,2022-05-07,CAVITE


In [ ]:
#Separating provinces in Luzon and adding it to a new dataframe
df_new = df_new[df_new["ProvRes"].str.contains("NCR|ABRA|ALBAY|APAYAO|AURORA|BATAAN|BATANGAS|BENGUET|BULACAN|CAGAYAN|CAMARINES NORTE|CAMARINES SUR|CAVITE|IFUGAO|ILOCOS NORTE|ILOCOS SUR|ISABELA|KALINGA|LA UNION|LAGUNA|MOUNTAIN PROVINCE|NUEVA ECIJA|NUEVA VIZCAYA|PAMPANGA|PANGASINAN|QUEZON|QUIRINO|RIZAL|SORSOGON|TARLAC|ZAMBALES")==True]

In [ ]:
cases_agg_df = df_new.groupby(["ProvRes", "DateRepConf"]).agg(['count']).sort_values(["ProvRes", "DateRepConf"], ascending = (True, True))
cases_agg_df = cases_agg_df.reset_index()
len(cases_agg_df)

19393

In [ ]:
cases_agg_df.tail()

,ProvRes,DateRepConf,CaseCode
,,,count
19388,ZAMBALES,2022-04-30,2
19389,ZAMBALES,2022-05-01,2
19390,ZAMBALES,2022-05-02,2
19391,ZAMBALES,2022-05-03,1
19392,ZAMBALES,2022-05-06,1


# **Community Mobility Section**
#### ```df_commob```

In [ ]:
df_commob = pd.read_csv("ph_commobility.csv")

In [ ]:
# Obtaining the NCR Values
df_commob[df_commob["sub_region_1"] == "National Capital Region"].tail()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
20759,PH,Philippines,National Capital Region,NaN,NaN,PH-00,NaN,ChIJbTgmYNLIlzMR0HiSrNoj7V8,2022-05-09,-22,25,-17,-42,-63,18
20760,PH,Philippines,National Capital Region,NaN,NaN,PH-00,NaN,ChIJbTgmYNLIlzMR0HiSrNoj7V8,2022-05-10,-21,23,-28,-23,-25,15
20761,PH,Philippines,National Capital Region,NaN,NaN,PH-00,NaN,ChIJbTgmYNLIlzMR0HiSrNoj7V8,2022-05-11,-20,26,-26,-27,-21,15
20762,PH,Philippines,National Capital Region,NaN,NaN,PH-00,NaN,ChIJbTgmYNLIlzMR0HiSrNoj7V8,2022-05-12,-19,24,-24,-26,-20,15
20763,PH,Philippines,National Capital Region,NaN,NaN,PH-00,NaN,ChIJbTgmYNLIlzMR0HiSrNoj7V8,2022-05-13,-16,29,-26,-27,-18,16


In [ ]:
# removing the unnecessary columns
df_commob = df_commob.drop(columns=['country_region_code', 'country_region', 'iso_3166_2_code', 'census_fips_code', 'place_id'])

In [ ]:
# changing to a datetime format
df_commob["date"] = pd.to_datetime(df_commob["date"], dayfirst=True, errors='coerce')
# df_commob["date"] = df_commob["date"].dt.strftime("%m-%d-%y")
# df_commob["date"] = pd.to_datetime(df_commob["date"], dayfirst=True, errors='coerce')

# renaming for shorter column names
df_commob.rename(columns={'retail_and_recreation_percent_change_from_baseline': 'retail_rec_baseline',
                          'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_pharma_baseline',
                          'parks_percent_change_from_baseline': 'parks_baseline',
                          'transit_stations_percent_change_from_baseline': 'transit_baseline',
                          'workplaces_percent_change_from_baseline': 'workplace_baseline',
                          'residential_percent_change_from_baseline': 'residental_baseline'}, inplace=True)

# changing to NCR to make it uniform
df_commob.loc[df_commob['sub_region_1'].str.contains('National Capital Region', case=False, na=False), 'sub_region_1'] = 'NCR'

In [ ]:
df_commob.dtypes

sub_region_1                       object
sub_region_2                       object
metro_area                         object
date                       datetime64[ns]
retail_rec_baseline                object
grocery_pharma_baseline            object
parks_baseline                     object
transit_baseline                   object
workplace_baseline                 object
residental_baseline                object
dtype: object

In [ ]:
# ensuring the provinces and date ranges all check out
df_commob = df_commob[(df_commob['date'] > "2020-3-16") & (df_commob['date'] < "2022-5-8")].reset_index(drop=True)  
df_commob = df_commob.sort_values(['sub_region_1', 'date'], ascending=(True, True)).reset_index(drop=True)

In [ ]:
# Capital Regions
df_commob["sub_region_1"].unique()

array(['Autonomous Region in Muslim Mindanao', 'Bicol', 'Cagayan Valley',
       'Calabarzon', 'Caraga', 'Central Luzon', 'Central Visayas',
       'Cordillera Administrative Region', 'Davao Region',
       'Eastern Visayas', 'Ilocos Region', 'MIMAROPA', 'NCR',
       'Northern Mindanao', 'SOCCSKSARGEN', 'Western Visayas',
       'Zamboanga Peninsula', nan], dtype=object)

# **Hospitals Section**
#### ```new_hosp_df```

In [ ]:
#renamed the DOH COVID Data Drop_date -05 DOH Data Collect
hospitals_df = pd.read_csv("DOH_Hospitals.csv")

# icu = icu beds with vacant or occupied status
# isolbed = isolation beds with vacant or occupied status 
# beds_ward = beds in coverted wards na vacant or occupied in status
# mechvent = mechanical ventilors that are either occupied or vacant 
# icu_v_nc = icu beds for non covid patients 
# mechvent_nc = mechanical venilators for non covid patients 

hospitals_df.head(2)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,updateddate,addeddate,reportdate,other_adm,doctor_adm,nurse_adm,qother,qdoctor,qnurse,isolbed_o,...,beds_ward_v,facilityname,region,province,city_mun,bgy,region_psgc,province_psgc,city_mun_psgc,bgy_psgc
0,2020-09-12 06:57:07,2020-09-12 06:57:07,2020-09-12,0.0,0.0,0.0,0.0,0.0,0.0,22.0,...,0.0,CAGDIANAO EVACUATION CENTER (MCCC 1),REGION XIII (CARAGA),DINAGAT ISLANDS,CAGDIANAO,POBLACION,PH160000000,PH168500000,PH168502000,PH168502008
1,2020-11-18 09:34:30,2020-11-18 09:34:30,2020-11-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,AGBALOGO BIU,REGION VI (WESTERN VISAYAS),AKLAN,MAKATO,AGBALOGO,PH060000000,PH060400000,PH060411000,PH060411001


In [ ]:
hospitals_df["reportdate"] = pd.to_datetime(hospitals_df["reportdate"], dayfirst=True, errors='coerce')

#hospitals_df["icu_beds"] = hospitals_df["icu_v"] + hospitals_df["icu_o"] # + hospitals_df["icu_v_nc"] + hospitals_df["icu_o_nc"] 
hospitals_df["isolation_beds"] = hospitals_df["isolbed_v"] + hospitals_df["isolbed_o"] 
hospitals_df["ward_beds"] = hospitals_df["beds_ward_v"] + hospitals_df["beds_ward_o"]
#hospitals_df["mech_ventilators"] = hospitals_df["mechvent_v"] + hospitals_df["mechvent_o"] # + hospitals_df["mechvent_v_nc"] + hospitals_df["mechvent_o_nc"] 

In [ ]:
hospitals_df.head(2)

,updateddate,addeddate,reportdate,other_adm,doctor_adm,nurse_adm,qother,qdoctor,qnurse,isolbed_o,...,region,province,city_mun,bgy,region_psgc,province_psgc,city_mun_psgc,bgy_psgc,isolation_beds,ward_beds
0,2020-09-12 06:57:07,2020-09-12 06:57:07,2020-09-12,0.0,0.0,0.0,0.0,0.0,0.0,22.0,...,REGION XIII (CARAGA),DINAGAT ISLANDS,CAGDIANAO,POBLACION,PH160000000,PH168500000,PH168502000,PH168502008,73.0,0.0
1,2020-11-18 09:34:30,2020-11-18 09:34:30,2020-11-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,REGION VI (WESTERN VISAYAS),AKLAN,MAKATO,AGBALOGO,PH060000000,PH060400000,PH060411000,PH060411001,5.0,0.0


In [ ]:
#new_hosp_df is the dataframe with only the relevant hospital data
new_hosp_df = pd.concat([hospitals_df["reportdate"], hospitals_df["province"], hospitals_df["isolation_beds"], hospitals_df["ward_beds"]],  axis=1)
new_hosp_df = new_hosp_df[new_hosp_df["province"].str.contains("NCR|ABRA|ALBAY|APAYAO|AURORA|BATAAN|BATANGAS|BENGUET|BULACAN|CAGAYAN|CAMARINES NORTE|CAMARINES SUR|CAVITE|IFUGAO|ILOCOS NORTE|ILOCOS SUR|ISABELA|KALINGA|LA UNION|LAGUNA|MOUNTAIN PROVINCE|NUEVA ECIJA|NUEVA VIZCAYA|PAMPANGA|PANGASINAN|QUEZON|QUIRINO|RIZAL|SORSOGON|TARLAC|ZAMBALES")==True]

In [ ]:
#Changed all the NCR names that are lengthy to just "NCR" 
#https://stackoverflow.com/questions/39768547/replace-whole-string-if-it-contains-substring-in-pandas
new_hosp_df.loc[new_hosp_df['province'].str.contains('NCR', case=False), 'province'] = 'NCR'

In [ ]:
new_hosp_df = new_hosp_df[(new_hosp_df['reportdate'] > "2020-03-16") & (new_hosp_df['reportdate'] < "2022-05-08")].reset_index(drop=True)  
new_hosp_df = new_hosp_df.groupby(["province", "reportdate"]).agg(['count']).sort_values(["province", "reportdate"], ascending = (True, True))
new_hosp_df = new_hosp_df.reset_index()
new_hosp_df.tail()

,province,reportdate,isolation_beds,ward_beds
,,,count,count
22135,ZAMBALES,2022-05-02,20,20
22136,ZAMBALES,2022-05-03,16,16
22137,ZAMBALES,2022-05-04,21,21
22138,ZAMBALES,2022-05-05,20,20
22139,ZAMBALES,2022-05-06,21,21


In [ ]:
new_hosp_df[new_hosp_df["province"] == "NCR"].tail()

,province,reportdate,isolation_beds,ward_beds
,,,count,count
15068,NCR,2022-05-02,38,38
15069,NCR,2022-05-03,38,38
15070,NCR,2022-05-04,38,38
15071,NCR,2022-05-05,38,38
15072,NCR,2022-05-06,38,38


# **COVID PH Lockdowns Section**
#### ```compiled_df```

In [ ]:
lockdowns_df = pd.read_csv("PH_lockdowns_cleanest.csv")
lockdowns_df["Start Date"] = pd.to_datetime(lockdowns_df["Start Date"])
lockdowns_df["End Date"] = pd.to_datetime(lockdowns_df["End Date"])
lockdowns_df.rename(columns={'Start Date': 'Start_Date', 'End Date': 'End_Date', 'Quarantine Type': 'Quarantine_Type'}, inplace=True)
lockdowns_df.dtypes

Province                   object
Quarantine_Type            object
Start_Date         datetime64[ns]
End_Date           datetime64[ns]
dtype: object

In [ ]:
lockdowns_df.tail()

,Province,Quarantine_Type,Start_Date,End_Date
167,TARLAC,MGCQ,2020-09-01,2022-05-07
168,ZAMBALES,ECQ,2020-03-17,2020-05-15
169,ZAMBALES,MECQ,2020-05-16,2020-05-31
170,ZAMBALES,GCQ,2020-06-01,2020-08-31
171,ZAMBALES,MGCQ,2020-09-01,2022-05-07


### Populating Function 

In [ ]:
# Creating a dataframe in the format of lockdown per day per province from the previous quarantine dataframe!

#https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/
# BETTER https://stackoverflow.com/questions/38741952/how-to-convert-data-of-type-panda-to-panda-dataframe

compiled_df = pd.DataFrame(columns = ["Province", "Quarantine_Type"])

for row in lockdowns_df.itertuples():
  holder_df = pd.DataFrame(columns = ["Province", "Quarantine_Type", "Start_Date", "End_Date"])
  holder_df.loc[holder_df.shape[0]] = list(row)[1:]
  
  idx = pd.date_range(holder_df.iloc[0]["Start_Date"], holder_df.iloc[0]["End_Date"])
  holder_df.set_index(holder_df.Start_Date, inplace=True)
  holder_df = holder_df.resample('D').sum().reindex(idx) 
  holder_df['Date'] = holder_df.index
  holder_df = holder_df.reset_index(drop=True)
  
  holder_df["Province"] = holder_df.iloc[0]["Province"]
  holder_df["Quarantine_Type"] = holder_df.iloc[0]["Quarantine_Type"]

  if row.Index == 0:
    compiled_df = holder_df
  else:
    compiled_df = compiled_df.append(holder_df, ignore_index=True)

In [ ]:
compiled_df.tail()

,Province,Quarantine_Type,Date
25019,ZAMBALES,MGCQ,2022-05-03
25020,ZAMBALES,MGCQ,2022-05-04
25021,ZAMBALES,MGCQ,2022-05-05
25022,ZAMBALES,MGCQ,2022-05-06
25023,ZAMBALES,MGCQ,2022-05-07


In [ ]:
# integer encoding the quarantine values from highest movement restrictions to lowest 
compiled_df = compiled_df.replace({'Quarantine_Type': {"ECQ": 4, "MECQ": 3, "GCQ": 2, "MGCQ": 1}})
compiled_df["Quarantine_Type"] = pd.to_numeric(compiled_df["Quarantine_Type"])

# **Incidence Rate Section**
#### ```populated_incidences_df```

In [ ]:
incidences_df = pd.read_csv("Luzon_Incidence_Rates_PopDens_2020.csv")
incidences_df["Start_Date"] = pd.to_datetime(incidences_df["Start_Date"])
incidences_df["End_Date"] = pd.to_datetime(incidences_df["End_Date"])

In [ ]:
incidences_df.dtypes

Province                      object
Population_Size                int64
Population_Density             int64
Start_Date            datetime64[ns]
End_Date              datetime64[ns]
dtype: object

In [ ]:
populated_incidences_df = pd.DataFrame(columns = ["Province", "Population_Size"])

for row in incidences_df.itertuples():
  holder_df = pd.DataFrame(columns = ["Province", "Population_Size", "Population_Density", "Start_Date", "End_Date"])
  holder_df.loc[holder_df.shape[0]] = list(row)[1:]
  
  idx = pd.date_range(holder_df.iloc[0]["Start_Date"], holder_df.iloc[0]["End_Date"])
  holder_df.set_index(holder_df.Start_Date, inplace=True)
  holder_df = holder_df.resample('D').sum().reindex(idx)
  holder_df['Date'] = holder_df.index
  holder_df = holder_df.reset_index(drop=True)
  
  holder_df["Province"] = holder_df.iloc[0]["Province"]
  holder_df["Population_Size"] = holder_df.iloc[0]["Population_Size"]
  holder_df["Population_Density"] = holder_df.iloc[0]["Population_Density"]

  if row.Index == 0:
    populated_incidences_df = holder_df
  else:
    populated_incidences_df = populated_incidences_df.append(holder_df, ignore_index=True)

In [ ]:
populated_incidences_df.tail()

,Province,Population_Size,Population_Density,Date
24237,ZAMBALES,909923.0,238.0,2022-05-03
24238,ZAMBALES,909923.0,238.0,2022-05-04
24239,ZAMBALES,909923.0,238.0,2022-05-05
24240,ZAMBALES,909923.0,238.0,2022-05-06
24241,ZAMBALES,909923.0,238.0,2022-05-07


# Main Dataframe Generating Loop!

In [ ]:
small_provs = ["ABRA", "ALBAY", "APAYAO", "AURORA", "BATAAN", "BATANGAS", "BENGUET", "BULACAN", "CAGAYAN", "CAMARINES NORTE", "CAMARINES SUR", "CAVITE", "IFUGAO", "ILOCOS NORTE", "ILOCOS SUR", 
             "ISABELA", "KALINGA", "LA UNION", "LAGUNA", "MOUNTAIN PROVINCE", "NCR", "NUEVA ECIJA", "NUEVA VIZCAYA", "PAMPANGA", "PANGASINAN", "QUEZON", "QUIRINO", "RIZAL", "SORSOGON", "TARLAC", "ZAMBALES"]

complete_provinces_data_df = pd.DataFrame(columns = ["Province"])

for i in small_provs:
  prov_name = i 
  prov_idx = pd.date_range('03-17-2020', '05-07-2022')

  #cases section
  cases_holder_df = cases_agg_df.loc[cases_agg_df['ProvRes'].str.contains(prov_name)].copy().reset_index(drop=True)
  cases_holder_df.set_index(cases_holder_df.DateRepConf, inplace=True)
  cases_holder_df = cases_holder_df.resample('D').sum().reindex(prov_idx).fillna(0)
  #cases_holder_df['smoothened_cases'] = cases_holder_df.CaseCode.ewm(alpha=0.3, adjust=False).mean()

  #recoveries section
  recover_holder_df = df_aggrecov.loc[df_aggrecov['ProvRes'].str.contains(prov_name)].copy().reset_index(drop=True)
  recover_holder_df.set_index(recover_holder_df.DateRecover, inplace=True)
  recover_holder_df = recover_holder_df.resample('D').sum().reindex(prov_idx).fillna(0)

  #deaths section
  death_holder_df = df_aggredied.loc[df_aggredied['ProvRes'].str.contains(prov_name)].copy().reset_index(drop=True)
  death_holder_df.set_index(death_holder_df.DateDied, inplace=True)
  death_holder_df = death_holder_df.resample('D').sum().reindex(prov_idx).fillna(0)
  
  #hospital section
  hosp_holder_df = new_hosp_df.loc[new_hosp_df['province'].str.contains(prov_name)].copy()
  hosp_holder_df.set_index(hosp_holder_df.reportdate, inplace=True)
  hosp_holder_df = hosp_holder_df.resample('D').sum().reindex(prov_idx).fillna(0)
  hosp_holder_df.insert(loc=0, column='Province', value=prov_name)

  #lockdown section 
  lock_holder_df = compiled_df.loc[compiled_df['Province'].str.contains(prov_name)].copy()
  lock_holder_df.set_index(lock_holder_df.Date, inplace=True)

  #incidence rates section
  incs_holder_df = populated_incidences_df.loc[populated_incidences_df['Province'].str.contains(prov_name)].copy()
  incs_holder_df.set_index(incs_holder_df.Date, inplace=True)

  #community mobility section
  if prov_name in "ALBAY|CAMARINES NORTE|CAMARINES SUR|SORSOGON":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Bicol', na=False)].copy()
  elif prov_name in "CAGAYAN|ISABELA|NUEVA VIZCAYA|QUIRINO":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Cagayan Valley', na=False)].copy()
  elif prov_name in "BATANGAS|CAVITE|LAGUNA|QUEZON|RIZAL":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Calabarzon', na=False)].copy()
  elif prov_name in "AURORA|BATAAN|BULACAN|NUEVA ECIJA|PAMPANGA|TARLAC|ZAMBALES":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Central Luzon', na=False)].copy()
  elif prov_name in "ABRA|APAYAO|BENGUET|IFUGAO|KALINGA|MOUNTAIN PROVINCE":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Cordillera Administrative Region', na=False)].copy()
  elif prov_name in "ILOCOS NORTE|ILOCOS SUR|LA UNION|PANGASINAN":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('Ilocos Region', na=False)].copy()
  elif prov_name in "NCR":
    com_mob_df = df_commob.loc[df_commob['sub_region_1'].str.contains('NCR', na=False)].copy()
  
  com_mob_df.set_index(com_mob_df.date, inplace=True)
  com_mob_df = com_mob_df.resample('D').sum().reindex(prov_idx).fillna(0)

  # Hosp Holder df is the dataframe everything else is appended to
  hosp_holder_df["covid_counts"] = cases_holder_df["CaseCode"]
  #hosp_holder_df["smoothened_covid_counts"] = cases_holder_df["smoothened_cases"]
  hosp_holder_df["recovery_counts"] = recover_holder_df["CaseCode"]
  hosp_holder_df["death_counts"] = death_holder_df["CaseCode"]
  hosp_holder_df["population_size"] = incs_holder_df["Population_Size"]
  #hosp_holder_df["population_density"] = incs_holder_df["Population_Density"]
  hosp_holder_df["incidence_rate"] = ((hosp_holder_df["covid_counts"]/incs_holder_df["Population_Size"]) * 100000).round(2)
  hosp_holder_df["quarantine_type_int"] = lock_holder_df["Quarantine_Type"]

  hosp_holder_df["retail_rec_baseline"] = com_mob_df["retail_rec_baseline"]
  hosp_holder_df["grocery_pharma_baseline"] = com_mob_df["grocery_pharma_baseline"]
  hosp_holder_df["parks_baseline"] = com_mob_df["parks_baseline"]
  hosp_holder_df["transit_baseline"] = com_mob_df["transit_baseline"]
  hosp_holder_df["workplace_baseline"] = com_mob_df["workplace_baseline"]
  hosp_holder_df["residental_baseline"] = com_mob_df["residental_baseline"]

  hosp_holder_df.insert(loc=0, column='Date', value = hosp_holder_df.index)

  #hosp_holder_df["smoothened_covid_counts"] = np.floor(hosp_holder_df["smoothened_covid_counts"])

  if i == "ABRA":
    complete_provinces_data_df = hosp_holder_df
  else:
    complete_provinces_data_df = complete_provinces_data_df.append(hosp_holder_df, ignore_index=True)

# insert susceptible counts after the DF is made bec computations can't be made while it's being modified 
complete_provinces_data_df.insert(
    loc = 8,
    column = 'susceptible_counts',
    value = complete_provinces_data_df["population_size"] - (complete_provinces_data_df["covid_counts"] + complete_provinces_data_df["recovery_counts"] + complete_provinces_data_df["death_counts"]))

/usr/local/lib/python3.7/dist-packages/pandas/core/apply.py:577: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  return f(*args, **kwargs)


In [ ]:
complete_provinces_data_df.tail(5)

,Date,Province,isolation_beds,ward_beds,covid_counts,recovery_counts,death_counts,population_size,susceptible_counts,incidence_rate,quarantine_type_int,retail_rec_baseline,grocery_pharma_baseline,parks_baseline,transit_baseline,workplace_baseline,residental_baseline
,,,count,count,,,,,,,,,,,,,
24237,2022-05-03,ZAMBALES,16.0,16.0,1.0,0.0,0.0,909923.0,909922.0,0.11,1,46,90,87,37,-22,22
24238,2022-05-04,ZAMBALES,21.0,21.0,0.0,0.0,0.0,909923.0,909923.0,0.00,1,44,82,71,38,2,18
24239,2022-05-05,ZAMBALES,20.0,20.0,0.0,0.0,0.0,909923.0,909923.0,0.00,1,44,87,65,37,4,18
24240,2022-05-06,ZAMBALES,21.0,21.0,1.0,0.0,0.0,909923.0,909922.0,0.11,1,37,87,70,37,8,19
24241,2022-05-07,ZAMBALES,0.0,0.0,0.0,0.0,0.0,909923.0,909923.0,0.00,1,47,95,70,58,32,13


In [ ]:
#complete_provinces_data_df.to_csv('Feb52022_all_provinces_complete_wComMobility.csv', index=False)

In [ ]:
# generate the NCR ONLY dataset
ncr_may72022 = complete_provinces_data_df[complete_provinces_data_df["Province"] == "NCR"].copy()
ncr_may72022.drop(['Province', 'population_size'], axis=1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [ ]:
ncr_may72022.tail(5)

,Date,isolation_beds,ward_beds,covid_counts,recovery_counts,death_counts,susceptible_counts,incidence_rate,quarantine_type_int,retail_rec_baseline,grocery_pharma_baseline,parks_baseline,transit_baseline,workplace_baseline,residental_baseline
,,count,count,,,,,,,,,,,,
16402,2022-04-18,39.0,39.0,77.0,5.0,2.0,13484378.0,0.57,3,-19,25,-20,-26,-22,14
16403,2022-04-19,39.0,39.0,67.0,4.0,1.0,13484390.0,0.50,3,-20,24,-24,-27,-21,16
16404,2022-04-20,39.0,39.0,124.0,0.0,0.0,13484338.0,0.92,3,-20,25,-23,-29,-21,16
16405,2022-04-21,39.0,39.0,53.0,3.0,1.0,13484405.0,0.39,3,-21,22,-22,-28,-21,16
16406,2022-04-22,39.0,39.0,80.0,5.0,1.0,13484376.0,0.59,3,-21,23,-23,-33,-18,17
16407,2022-04-23,39.0,39.0,72.0,1.0,2.0,13484387.0,0.53,3,-12,32,-14,-17,-4,10
16408,2022-04-24,39.0,39.0,97.0,3.0,2.0,13484360.0,0.72,3,-13,30,-9,-11,4,8
16409,2022-04-25,39.0,39.0,95.0,4.0,2.0,13484361.0,0.70,3,-18,26,-18,-27,-20,15
16410,2022-04-26,39.0,39.0,39.0,8.0,1.0,13484414.0,0.29,3,-19,26,-26,-27,-20,16


In [ ]:
ncr_may72022.to_csv('ncr_may72022.csv', index=False)